# Desafío

En este desafío, el servidor permite a un usuario registrarse con un correo electrónico y un conjunto de datos adicionales, y devuelve un perfil cifrado con AES en modo CBC.

Si el usuario registró el correo usuario@example.com y puso como datos “Juan P. Usuario”, el perfil tendrá la forma:

user=usuario@example.com;data=Juan P. Usuario;role=user

```
user=usuario@example.com;data=Juan P. Usuario;role=user
```


Es decir, está compuesto por un conjunto de pares atributo-valor, de la forma <atributo>=<valor>, y separados por punto y coma.

El servidor devuelve el perfil cifrado, precedido por un IV de 16 bytes, y codificado en base64.

El desafío es alterar el mensaje cifrado de manera tal que al descifrarlo incluya el par role=admin. Esta alteración es posible porque el modo CBC es maleable. Es posible modificar el texto cifrado de manera tal que al descifrarlo el texto claro haya sido modificado de manera (parcialmente) predecible.

Este tipo de ataque se denomina bit flipping, porque el cambio de un bit en el texto cifrado provoca un cambio el bit correspondiente del siguiente bloque de texto claro (aunque altera completamente el texto claro del bloque modificado).

Para obtener los textos cifrados, se debe hacer un requerimiento POST a una URL de la forma:

`https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/<email>/register`
        
dónde <email> debe ser reemplazado por una dirección de correo electrónico registrada.

El contenido debe ser de tipo FORM, con dos campos:

Un campo email, con una dirección de correo a registrar. No es necesario que sea la misma dirección utilizada para acceder al desafío.
Un campo data, con datos arbitrarios, codificado en base64. Los datos no pueden contener el carácter ‘;’ ni el carácter ‘=’.
La respuesta será un perfil como el descripto, cifrado, concatenado con el IV y codificado en base64.

Si en lugar de hacer un requerimiento POST se hace un requerimiento GET, muestra un formulario que permite cargar los campos requeridos.

# Análisis
La clave para resolver este desafío consiste en entender el comportamiento del modo CBC ante modificaciones del texto cifrado. El resultado de descifrar el bloque modificado es impredecible, pero el siguiente bloque será modificado en exactamento los mismos bits que fueron cambiados. Como en este caso el contenido del campo data es arbitrario, cualquier modificación de su contenido no será detectada, por lo que la solución es escoger un valor para data que tenga la longitud adecuada, de manera que sepamos en qué bloque están los datos a modificar, y alterar el bloque anterior de manera apropiada.

# Puntos a tener en cuenta
* El algoritmo utilizado es AES, por lo que los bloques son de 128 bits (16 bytes).
* El texto cifrado enviado como respuesta debe descifrarse a una consulta válida, con pares atributo-valor separados por punto y coma:
  * Debe tener exactamente un campo user, con una dirección de correo válida.
  * Debe tener exactamente un campo role con el valor admin.
* Como el modo utilizado es CBC, se utiliza relleno (padding). El relleno utilizado es el estandarizado en PKCS7: se aplican tantos bytes como sea necesario para que el mensaje sea múltiplo del tamaño de bloque, y el valor de esos bytes es la longitud del relleno. Si el mensaje original era múltiplo del tamaño de bloque, el relleno es un bloque adicional con todos sus bytes con el valor 16. Al descifrar el mensaje, el relleno debe ser válido.

In [ ]:
import requests
import base64

In [5]:
def b64_decode(message_encrypted):
  message_decoded_from_base64 = base64.b64decode(message_encrypted)
  return message_decoded_from_base64.hex()

In [6]:
def print_block(lista, bytes_):
  hex_list = ''
  for each in range(0,len(lista)):
    if each % bytes_ == 0:
      print(hex_list)
      hex_list = ''
    hex_list = hex_list + ' ' + lista[each:each+1]
  print(hex_list)

In [7]:
def print_hex(lista, bytes_):
  hex_list = ''
  for each in range(0,len(lista), 2):
    if each % bytes_ == 0:
      print(hex_list)
      hex_list = ''
    hex_list = hex_list + ' ' + lista[each:each+2]
  print(hex_list)

In [8]:
def print_hex_list(lista, bytes_):
  hex_list = []
  for each in range(0,len(lista)):
    if each != 0 and each % bytes_ == 0:
      print(hex_list)
      hex_list = []
    hex_list.append(lista[each])
  print(hex_list)

In [9]:
def get_profile(email_register, data):
  profile = f'user={email_register};data={data};role=user'
  return profile

In [10]:
def xor_bytes(a: bytes, b: bytes) -> bytes:
    if len(a) != len(b):
        raise ValueError("Cannot xor non-equal length bytes")

    result = bytearray(a)

    for i, byte in enumerate(b):
      result[i] ^= byte
    return bytes(result)

In [ ]:
email = 'solujan@gmail.com'
email_register = 'solujan@gmail.com'
data = 'Sof!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'
url = f'https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{email}/register'

payload = {'email':email,'data':base64.b64encode(data.encode('ascii'))}

# Solicitud
request = requests.post(url, files=payload)
# Body de la respuesta
message_encoded_in_base64 = request.content
message_encoded_in_base64

b't9IwmewqBF5gX9hKwK56PhLQUiQhvrCv9EwNmRvo/eUG0xq1MXfJr5cfX/GPHLWrz0AYBuvACCin7OCOoMpbkCeupzUPY681asPwSaGDy13axgPcPgHpvVNwOl7S4ZpY'

In [ ]:
profile = get_profile(email_register, data)
print(profile)

user=solujan@gmail.com;data=Sof!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!;role=user


In [ ]:
print_block(profile, 16)


 u s e r = s o l u j a n @ g m a
 i l . c o m ; d a t a = S o f !
 ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
 ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ;
 r o l e = u s e r


In [ ]:
message_decoded_hex = b64_decode(message_encoded_in_base64)
message_decoded_hex

'b7d23099ec2a045e605fd84ac0ae7a3e12d0522421beb0aff44c0d991be8fde506d31ab53177c9af971f5ff18f1cb5abcf401806ebc00828a7ece08ea0ca5b9027aea7350f63af356ac3f049a183cb5ddac603dc3e01e9bd53703a5ed2e19a58'

In [ ]:
iv = message_decoded_hex[:32]
cipher_text = message_decoded_hex[32:]

In [ ]:
print_hex(iv, 32)
print_hex(cipher_text, 32)


 b7 d2 30 99 ec 2a 04 5e 60 5f d8 4a c0 ae 7a 3e

 12 d0 52 24 21 be b0 af f4 4c 0d 99 1b e8 fd e5
 06 d3 1a b5 31 77 c9 af 97 1f 5f f1 8f 1c b5 ab
 cf 40 18 06 eb c0 08 28 a7 ec e0 8e a0 ca 5b 90
 27 ae a7 35 0f 63 af 35 6a c3 f0 49 a1 83 cb 5d
 da c6 03 dc 3e 01 e9 bd 53 70 3a 5e d2 e1 9a 58


![](https://bernardoamc.com/0040327c3dde91c400bdb385c38099e8/cbc_decryption.svg)

In [ ]:
ctx = bytes.fromhex(cipher_text)
ctx

b"\x12\xd0R$!\xbe\xb0\xaf\xf4L\r\x99\x1b\xe8\xfd\xe5\x06\xd3\x1a\xb51w\xc9\xaf\x97\x1f_\xf1\x8f\x1c\xb5\xab\xcf@\x18\x06\xeb\xc0\x08(\xa7\xec\xe0\x8e\xa0\xca[\x90'\xae\xa75\x0fc\xaf5j\xc3\xf0I\xa1\x83\xcb]\xda\xc6\x03\xdc>\x01\xe9\xbdSp:^\xd2\xe1\x9aX"

In [ ]:
a_block = b'!' * AES.block_size
flipper = xor_bytes(a_block, b';role=admin;'.ljust(AES.block_size, b'\x00'))
padded = flipper.rjust(AES.block_size*3, b'\x00').ljust(len(ctx), b'\x00')
print(f"Padded: {len(list(padded))}")
print_hex(padded.hex(),32)

new_cipher = xor_bytes(ctx, padded)
print(f"\nNew_cipher: {len(list(new_cipher))}")
print_hex(new_cipher.hex(),32)

Padded: 80

 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 1a 53 4e 4d 44 1c 40 45 4c 48 4f 1a 21 21 21 21
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00

New_cipher: 80

 12 d0 52 24 21 be b0 af f4 4c 0d 99 1b e8 fd e5
 06 d3 1a b5 31 77 c9 af 97 1f 5f f1 8f 1c b5 ab
 d5 13 56 4b af dc 48 6d eb a4 af 94 81 eb 7a b1
 27 ae a7 35 0f 63 af 35 6a c3 f0 49 a1 83 cb 5d
 da c6 03 dc 3e 01 e9 bd 53 70 3a 5e d2 e1 9a 58


In [ ]:
print(f'iv: {iv} {type(iv)}\nnew cipher: {new_cipher.hex()}')

iv: b7d23099ec2a045e605fd84ac0ae7a3e <class 'str'>
new cipher: 12d0522421beb0aff44c0d991be8fde506d31ab53177c9af971f5ff18f1cb5abd513564bafdc486deba4af9481eb7ab127aea7350f63af356ac3f049a183cb5ddac603dc3e01e9bd53703a5ed2e19a58


In [ ]:
b64 = base64.b64encode(bytes.fromhex(iv + new_cipher.hex())).decode()
b64

't9IwmewqBF5gX9hKwK56PhLQUiQhvrCv9EwNmRvo/eUG0xq1MXfJr5cfX/GPHLWr1RNWS6/cSG3rpK+Uget6sSeupzUPY681asPwSaGDy13axgPcPgHpvVNwOl7S4ZpY'

In [ ]:
f = open("message.txt", "w")
f.write(b64)
f.close()

In [ ]:
url_answ = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/cbc-bitflip/{email}/answer"

In [ ]:
files = {'message': open('message.txt','rb')}
res = requests.post(url_answ, files=files)
res.text

'¡Ganaste!\n'

## CIPHER


In [2]:
!pip install pycryptodome



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.6 MB/s eta 0:00:00


In [3]:
from base64 import b64decode
from base64 import b64encode

from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
IV = get_random_bytes(AES.block_size)

class AESCipher:
    def __init__(self, key):
        self.key = key
        self.iv = IV

    def encrypt(self, data):
        self.cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        return self.cipher.encrypt(pad(data.encode('utf-8'), AES.block_size))

    def decrypt(self, data):
        self.cipher = AES.new(self.key, AES.MODE_CBC, self.iv)
        return unpad(self.cipher.decrypt(data), AES.block_size)

In [16]:
key = b'laclavesecreta12'
msg = "user=solujan@gmail.com;data=Sof!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!;role=user"
print_block(msg, 16)


print(f'\nOriginal Message: {msg}, len {len(msg)}')


ctx = AESCipher(key).encrypt(msg)
print(f'\nCiphertext      :{len(ctx)}')
print_hex(ctx.hex(),32)


a_block = b'!' * AES.block_size
print(a_block)
flipper = xor_bytes(a_block, b';role=admin;'.ljust(AES.block_size, b'\x00'))
print(f'\nflipper    :{len(flipper)}')
print_hex(flipper.hex(),32)
padded = flipper.rjust(AES.block_size*3, b'\x00').ljust(len(ctx), b'\x00')
print(f'\nBit attack    :{len(padded)}')
print_hex(padded.hex(),32)

new_ct = xor_bytes(ctx, padded)
print(f'\nnew_ct    :{len(new_ct)}')
print_hex(new_ct.hex(),32)
ptext = AESCipher(key).decrypt(new_ct)
print(f'\nPlain Text     : {len(ptext)}')
print_hex(ptext.hex(),32)
print(f'\nOutput     : {ptext}')



 u s e r = s o l u j a n @ g m a
 i l . c o m ; d a t a = S o f !
 ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! !
 ! ! ! ! ! ! ! ! ! ! ! ! ! ! ! ;
 r o l e = u s e r

Original Message: user=solujan@gmail.com;data=Sof!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!;role=user, len 73

Ciphertext      :80

 06 17 86 a6 4a f3 13 b4 8c ad e2 20 50 8f 38 73
 00 00 4b 1d e4 4b 6c c0 30 0b 83 e1 3c 39 ed f0
 8b 3c 8c da b5 fc 4d 69 5c 0d 1c 75 fa 5f 43 b0
 b1 25 de 65 0b 73 dc a2 92 c2 61 3b 2b c3 26 ae
 62 19 19 ef 45 e3 2c 56 12 b5 f2 f8 27 54 fa 7b
b'!!!!!!!!!!!!!!!!'

flipper    :16

 1a 53 4e 4d 44 1c 40 45 4c 48 4f 1a 21 21 21 21

Bit attack    :80

 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 1a 53 4e 4d 44 1c 40 45 4c 48 4f 1a 21 21 21 21
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00

new_ct    :80

 06 17 86 a6 4a f3 13 b4 8c ad e2 20 50 8f 38 73
 00 00 4b 1d e4 4b 6c c0 30 0b 83 e1 3c 39 ed f0
 91 6f c2 97 f1

In [17]:
b';role=admin;'.ljust(AES.block_size, b'\x00')

b';role=admin;\x00\x00\x00\x00'